# What is Selenium?

## 1

What environment do we use?

See [What is Docker?](What%20is%20Docker.ipynb)

Run Selenium in Docker?


In [1]:
var path = require('path');
var corePath = path.resolve(path.join(__dirname, '..', 'Core'));
var importer = require(corePath);
var writeFileSync = require('fs').writeFileSync;

// add some run commands to the bash script
var bashToRun = (code) => code.split('\n').map(l => 'RUN ' + l)
    .join('\n').replace(/\\\s*\nRUN\s*/ig, '\\\n ');

// create a selenium Dockerfile with a vnc connection
var interpretAllPath = path.join('..', 'Core', 'How to interpret all notebooks.ipynb');
var seleniumDocker = (outputFile) => {
    return importer.importNotebook(interpretAllPath)
        .then((f) => {
            return Promise.all([
            interpret('run selenium'),
            // add some extra services
            interpret('linux dev tools'),
            interpret('vnc html'),
            interpret('vnc docker')
        ])})
        .then(r => r.map(r => r[0]))
        .then(r => interpretObject(r))
        .then(r => {
            // lets fix the middle result
            r[1].code = bashToRun(r[1].code);
            r[2].code = bashToRun(r[2].code);
            return r.map(r => r.code).join('\n');
        })
        .then(r => {
            // save the Dockerfile
            writeFileSync(outputFile, r);
            return r;
        });
};
(seleniumDocker);


[Function: seleniumDocker]

How to set up selenium server?

In [2]:
var path = require('path');
var execSync = require('child_process').execSync;
$$.async();
var getSeleniumServer = () => {
    return seleniumDocker(path.join(__dirname, 'Dockerfile'))
        .then(r => execSync('docker build -t act-selenium .').toString())
        .then(r => {
            if(execSync('docker ps').indexOf('act-selenium') !== -1) {
                return r + execSync('docker rm act-selenium').toString();
            }
        })
        .then(r => r + execSync('docker run --user=seluser --shm-size=2g --name act-selenium -d -p 8888:8888 -p 6080:6080 -p 5900:5900 -p 4444:4444 -p 4200:4200 -p 3000:3000 -v /Users/briancullinan/Documents/robots/bots/defaultProfile:/usr/profile -v /Users/briancullinan/Downloads:/data/downloads act-selenium').toString());
};
(getSeleniumServer);


<Buffer 39 64 62 38 65 62 36 65 33 31 36 34 36 34 34 31 38 64 39 38 33 66 38 34 35 32 66 63 31 33 37 66 38 63 65 63 34 30 37 33 39 61 31 33 34 66 35 64 35 30 ... >

In [ ]:
$$.async();
getSeleniumServer()
    .then(r => $$.sendResult(r))
    .catch(r => $$.sendError(r));

# 2

How to connect to VNC through the web browser?

[Click here to open](http://localhost:6080/vnc.html?password=secret&host=localhost&port=6080&autoconnect=true&resize=downscale&view_only=true) in a separate browser


In [6]:
var vncIframe = () => `
<div style="display:block; width:100%; padding-bottom:75%;position:relative;">
<iframe id="vnc"
style="position:absolute;top:0;right:0;bottom:0;left:0;width:100%;height:100%;border:0;" 
src="http://localhost:6080/vnc.html?password=secret&amp;host=localhost&amp;port=6080&amp;autoconnect=true&amp;resize=downscale&amp;view_only=true"></iframe></div>
`;
(vncIframe);

In [8]:
$$.mime({'text/html': vncIframe()});

## 3

How to run a selenium test on the Docker machine?
